In [1]:
import yfinance as yf

# Download financial data for a company (e.g., Apple Inc.)
ticker = 'RTX'
company = yf.Ticker(ticker)

# Download financial statements
income_statement = company.financials.T
balance_sheet = company.balance_sheet.T
cash_flow = company.cashflow.T

In [9]:
# Calculate Free Cash Flow (FCF)
# Operating Cash Flow = 'Total Cash From Operating Activities'
# Capital Expenditures = 'Capital Expenditures'
operating_cash_flow = cash_flow['Operating Cash Flow']
capital_expenditure = cash_flow['Capital Expenditure']
FCF = operating_cash_flow + capital_expenditure #plus bcuz capex is negative in data
print(FCF)
print(cash_flow["Free Cash Flow"])


2023-12-31    11049000000.0
2022-12-31     9943000000.0
2021-12-31     9513000000.0
2020-12-31     5573000000.0
2019-12-31              NaN
dtype: object
2023-12-31    4717000000.0
2022-12-31    4393000000.0
2021-12-31    4629000000.0
2020-12-31    1639000000.0
2019-12-31             NaN
Name: Free Cash Flow, dtype: object


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

# Function to fetch financial data
def fetch_financial_data(ticker):
    """
    Fetches financial data for a given ticker using yfinance.

    Parameters:
    ticker (str): Stock ticker symbol.

    Returns:
    dict: Dictionary containing financial statements (income, balance, cashflow) and info.
    """
    company = yf.Ticker(ticker)
    financial_data = {
        'income_statement': company.financials.T,
        'balance_sheet': company.balance_sheet.T,
        'cash_flow': company.cashflow.T,
        'info': company.info
    }
    return financial_data

# Function to extract and clean FCF data
def extract_fcf(cash_flow, fcf_column='Free Cash Flow'):
    """
    Extracts Free Cash Flow (FCF) from the cash flow statement.

    Parameters:
    cash_flow (DataFrame): Cash flow statement data.
    fcf_column (str): Column name for Free Cash Flow. Default is 'Free Cash Flow'.

    Returns:
    Series: Cleaned Free Cash Flow data.
    """
    if fcf_column in cash_flow.columns:
        fcf = cash_flow[fcf_column].dropna()
        return fcf
    else:
        raise ValueError(f"{fcf_column} column is missing from the cash flow statement.")

# Function to calculate CAGR (Compound Annual Growth Rate)
def calculate_cagr(historical_fcf):
    """
    Calculates the Compound Annual Growth Rate (CAGR) based on historical FCF data.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow data.

    Returns:
    float: CAGR value.
    """
    start_value = historical_fcf.iloc[0]
    end_value = historical_fcf.iloc[-1]
    periods = len(historical_fcf) - 1
    
    if start_value <= 0 or end_value <= 0:
        raise ValueError("FCF values must be positive for CAGR calculation.")
    
    cagr = (end_value / start_value)**(1 / periods) - 1
    return cagr

# Function to calculate historical average growth rate
def calculate_average_growth_rate(historical_fcf):
    """
    Calculates the average annual growth rate based on historical FCF data.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow data.

    Returns:
    float: Average annual growth rate.
    """
    growth_rates = historical_fcf.pct_change().dropna()
    avg_growth_rate = growth_rates.mean()
    return avg_growth_rate

# Function to forecast FCF using calculated growth rate
def forecast_fcf(historical_fcf, growth_rate, years=5):
    """
    Forecasts future FCF based on historical FCF and a calculated growth rate.

    Parameters:
    historical_fcf (Series): Historical Free Cash Flow.
    growth_rate (float): Calculated growth rate for forecasting.
    years (int): Number of years to forecast. Default is 5 years.

    Returns:
    list: Forecasted FCF for the specified years.
    """
    last_fcf = historical_fcf[-1]
    fcf_forecast = [last_fcf * (1 + growth_rate)**i for i in range(1, years + 1)]
    return fcf_forecast

# Function to calculate the terminal value
def calculate_terminal_value(last_fcf, terminal_growth_rate=0.02, discount_rate=0.10):
    """
    Calculates the terminal value using the Gordon Growth Model.

    Parameters:
    last_fcf (float): Last projected Free Cash Flow.
    terminal_growth_rate (float): Growth rate for terminal value. Default is 2%.
    discount_rate (float): Discount rate (WACC). Default is 10%.

    Returns:
    float: Terminal Value.
    """
    terminal_value = last_fcf * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    return terminal_value

# Function to calculate the present value of future cash flows
def discount_cash_flows(fcf_forecast, terminal_value, discount_rate):
    """
    Calculates the present value of forecasted FCFs and terminal value.

    Parameters:
    fcf_forecast (list): Forecasted FCF values.
    terminal_value (float): Terminal value.
    discount_rate (float): Discount rate (WACC).

    Returns:
    float: Total discounted cash flow value.
    """
    discounted_fcf = [fcf / (1 + discount_rate)**i for i, fcf in enumerate(fcf_forecast, 1)]
    discounted_terminal_value = terminal_value / (1 + discount_rate)**len(fcf_forecast)
    return sum(discounted_fcf) + discounted_terminal_value

# Main DCF function with automatic growth rate calculation
def dcf_analysis(ticker, terminal_growth_rate=0.02, discount_rate=0.10, growth_rate_method='cagr'):
    """
    Performs DCF analysis for a given stock ticker using calculated growth rate based on historical FCF data.

    Parameters:
    ticker (str): Stock ticker symbol.
    terminal_growth_rate (float): Growth rate for terminal value. Default is 2%.
    discount_rate (float): Discount rate (WACC). Default is 10%.
    growth_rate_method (str): Method to calculate growth rate ('cagr' or 'average'). Default is 'cagr'.

    Returns:
    float: Intrinsic value per share.
    """
    # Fetch financial data
    financial_data = fetch_financial_data(ticker)
    cash_flow = financial_data['cash_flow']
    
    # Extract Free Cash Flow (FCF)
    historical_fcf = extract_fcf(cash_flow)
    
    # Calculate growth rate based on method
    if growth_rate_method == 'cagr':
        growth_rate = calculate_cagr(historical_fcf)
    elif growth_rate_method == 'average':
        growth_rate = calculate_average_growth_rate(historical_fcf)
    else:
        raise ValueError("Invalid growth rate calculation method. Choose 'cagr' or 'average'.")
    
    print(f"Calculated Growth Rate: {growth_rate:.4f}")
    
    # Forecast future FCF
    fcf_forecast = forecast_fcf(historical_fcf, growth_rate)
    
    # Calculate terminal value
    terminal_value = calculate_terminal_value(fcf_forecast[-1], terminal_growth_rate, discount_rate)
    
    # Discount FCFs and terminal value
    total_pv = discount_cash_flows(fcf_forecast, terminal_value, discount_rate)
    
    # Calculate enterprise value
    enterprise_value = total_pv
    
    # Calculate equity value (assuming net debt)
    balance_sheet = financial_data['balance_sheet']
    net_debt = balance_sheet['Total Debt'].iloc[-1] - balance_sheet['Cash'].iloc[-1]
    equity_value = enterprise_value - net_debt
    
    # Intrinsic value per share
    shares_outstanding = financial_data['info']['sharesOutstanding'] / 1e6  # in million
    intrinsic_value_per_share = equity_value / shares_outstanding
    
    return intrinsic_value_per_share

# Run DCF analysis for a given stock ticker using calculated growth rate
if __name__ == '__main__':
    ticker = 'AAPL'  # Example ticker
    intrinsic_value = dcf_analysis(ticker, growth_rate_method='cagr')
    print(f"Intrinsic Value per Share for {ticker}: ${intrinsic_value:.2f}")
